# Variables

In previous tutorials, all of our skills used the single default variable {{$input}}. For a lot of scenarios, this single input variable is just fine, but Semantic Kernel supports an SKContext object which acts a key-value pair to hold many variables when necessary. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.15.230531.5-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

// load settings specific to you!
MySettings settings = Settings.LoadFromFile();

// configure AI backend used by the kernel
var builder = new KernelBuilder();
if (settings.Type == "azure")
    builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.WithOpenAITextCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

The SKContext is a Dictionary<string, TrustAwareString>. For this tutorial, we'll just treat a TrustAwareString like any other string of characters. The first string in the dictionary represents the name of a variable and the second is the value of the variable to be used by the function. You can add many of these key-value pairs and provide the entire SKContext when invoking the function.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

// define your inline function using two variables - and opts to not use the default 'input' variable
string functionDefinition = """
solve for x and y: 
{{$equation1}}
{{$equation2}}
""";
var solveFunction = kernel.CreateSemanticFunction(functionDefinition);

// Initialize your context
SKContext context = kernel.CreateNewContext();
context["equation1"] = "x + y = 7";
context["equation2"] = "x - y = 3";

var completion = await solveFunction.InvokeAsync(context);

Console.WriteLine(completion);

After running the code above, you can see how the two variables (equation1 and equation2) are replaced by the values stored in the SKContext object. First, the two variable names are inserted into the inline function. Later an SKContext object is created through the CreateNewContext method on the kernel object. Subsequently, an entry is made in the dictionary for each key-value pair. (If you are unfamiliar with dictionaries, think of them like a list of items where each item is defined by a unique 'key' and it's corresponding 'value'. In the code above one item in the dictionary has a key equal to "equation1" and it's corresponding value is "x + y = 7".)

What if I don't know the number of variables I might have until I run the application? In the next example, we'll build a function definition more dynamically.

In [13]:
using Microsoft.SemanticKernel.Orchestration;

// define your inline function
string functionDefinition = string.Empty;

// Initialize your context
SKContext context = kernel.CreateNewContext();
context["fact1"] = "Five people ran a race.";
context["fact2"] = "Al finished before Bob, but behind Cal.";
context["fact3"] = "Dia finished before Ely, but behind Bob.";
context["question"] = "In what order did each finish the race?";

foreach (var v in context.Variables)
{
    functionDefinition += $"{{${v.Value}}} ";
};

var solveFunction = kernel.CreateSemanticFunction(functionDefinition);

var completion = await solveFunction.InvokeAsync(context);

Console.WriteLine(completion);



The order of finish in the race was: Cal, Al, Bob, Dia, Ely.


As you can see above, you aren't limited by one, two - or even four variables in your function. (You are limited by tokens, but more on that and prompt engineering in another tutorial.)

You can see we first built an SKContext variable and adding several facts to that dictionary. The variable names are replaced by the actual strings so it doesn't really matter what the dictionary key is so long as it makes sense to you the developer!

We then iterated through the SKContext object's Variables property to append each string together separated by a single space. The rest of the code runs like any function.